In [1]:
#for google colab
from google.colab import drive
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark
import os
!mkdir -p drive

Mounted at /content/drive
tar: spark-3.1.2-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 58.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=99e055bcdbecf84fc79011328bac7237d1745394be1a012ba007902457c3f929
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [10]:
# import some libraries
import pandas as pd
import numpy as np
import findspark
findspark.init()
import pyspark
import os
import urllib
import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.feature import *

In [31]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [5]:
sc = pyspark.sql.SparkSession.builder.appName('Test_for_IRIS').getOrCreate()

In [6]:
csv_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
iris =  pd.read_csv(csv_url, names = col_names)

In [8]:
#Load data from Iris.
df = sc.createDataFrame(iris)
df.show()

+------------+-----------+------------+-----------+-----------+
|Sepal_Length|Sepal_Width|Petal_Length|Petal_Width|      Class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4| 

In [11]:
feature_cols = df.columns[:-1]
#feature enginering
assembler = pyspark.ml.feature.VectorAssembler(inputCols=feature_cols, outputCol='features')
df = assembler.transform(df)

In [13]:
df.show()

+------------+-----------+------------+-----------+-----------+-----------------+
|Sepal_Length|Sepal_Width|Petal_Length|Petal_Width|      Class|         features|
+------------+-----------+------------+-----------+-----------+-----------------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|[4.4,2.9,1.4,0.2]|
|         4.9|  

In [14]:
df = df.select(['features','class'])
label = pyspark.ml.feature.StringIndexer(inputCol = 'class', outputCol = 'label').fit(df)
df = label.transform(df)
df = df.select(['features', 'label'])
df.show()

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|  0.0|
|[4.9,3.0,1.4,0.2]|  0.0|
|[4.7,3.2,1.3,0.2]|  0.0|
|[4.6,3.1,1.5,0.2]|  0.0|
|[5.0,3.6,1.4,0.2]|  0.0|
|[5.4,3.9,1.7,0.4]|  0.0|
|[4.6,3.4,1.4,0.3]|  0.0|
|[5.0,3.4,1.5,0.2]|  0.0|
|[4.4,2.9,1.4,0.2]|  0.0|
|[4.9,3.1,1.5,0.1]|  0.0|
|[5.4,3.7,1.5,0.2]|  0.0|
|[4.8,3.4,1.6,0.2]|  0.0|
|[4.8,3.0,1.4,0.1]|  0.0|
|[4.3,3.0,1.1,0.1]|  0.0|
|[5.8,4.0,1.2,0.2]|  0.0|
|[5.7,4.4,1.5,0.4]|  0.0|
|[5.4,3.9,1.3,0.4]|  0.0|
|[5.1,3.5,1.4,0.3]|  0.0|
|[5.7,3.8,1.7,0.3]|  0.0|
|[5.1,3.8,1.5,0.3]|  0.0|
+-----------------+-----+
only showing top 20 rows



In [26]:
#knn
#VectorAssembler之后collect的list没有办法直接应用在KNeighborsClassifier，需要reshape一下。
from sklearn.neighbors import KNeighborsClassifier

train, test = df.randomSplit([0.70, 0.30])
features_train = train.select('features').collect()
features_train = np.array(features_train).reshape(len(np.array(features_train)),-1)
label_train = train.select('label').collect()
neight = KNeighborsClassifier(n_neighbors = 5)
neight.fit(features_train, label_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [27]:
features_test = test.select('features').collect()
features_test = np.array(features_test).reshape(len(np.array(features_test)),-1)
label_test = test.select('label').collect()

In [32]:
neighbors = neight.kneighbors(features_test, return_distance = False)

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(neight.predict(features_test),label_test)

0.9642857142857143

In [48]:
#Logistic regression
reg = 0.05

lr = pyspark.ml.classification.LogisticRegression(regParam = reg)
model = lr.fit(train)
prediction = model.transform(test)
print('Prediction')
prediction.show()

Prediction
+-----------------+-----+--------------------+--------------------+----------+
|         features|label|       rawPrediction|         probability|prediction|
+-----------------+-----+--------------------+--------------------+----------+
|[4.3,3.0,1.1,0.1]|  0.0|[3.39486301700089...|[0.93066421428220...|       0.0|
|[4.4,3.0,1.3,0.2]|  0.0|[3.10457130708884...|[0.90981081703082...|       0.0|
|[4.4,3.2,1.3,0.2]|  0.0|[3.36041681207839...|[0.94139846150402...|       0.0|
|[4.6,3.2,1.4,0.2]|  0.0|[3.15765758574732...|[0.92558738556651...|       0.0|
|[4.6,3.4,1.4,0.3]|  0.0|[3.29971961828427...|[0.94772933535664...|       0.0|
|[4.8,3.0,1.4,0.3]|  0.0|[2.63535513150339...|[0.85324953337981...|       0.0|
|[4.8,3.4,1.6,0.2]|  0.0|[3.16065811487652...|[0.93592668429120...|       0.0|
|[4.9,2.4,3.3,1.0]|  1.0|[0.04336832990942...|[0.21489294867091...|       1.0|
|[4.9,3.0,1.4,0.2]|  0.0|[2.67280186555509...|[0.85113808551306...|       0.0|
|[5.0,2.0,3.5,1.0]|  1.0|[-0.644830917529

In [81]:
accuracy_score(np.array(prediction.select('label').collect()).reshape(1,-1).flatten(), np.array(prediction.select('prediction').collect()).reshape(1,-1).flatten())

0.9107142857142857